In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
!pip install Sastrawi

In [6]:
!pip install xlrd

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [7]:
# Random seed for consistency
np.random.seed(42)

nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ali
[nltk_data]     Hakim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Label Encoder use to Encode target labels with value between 0 and n_classes-1
Encoder = LabelEncoder()

# TfidfVectorizer Convert a collection of raw documents to a matrix of TF-IDF features.
Tfidf_vect = TfidfVectorizer()

In [33]:
# Read dataset using pandas function read_csv

data_train = pd.read_csv('bigdata_dataset.csv')
data_train = data_train.dropna()


# data_train=data_train.astype(str)

In [34]:
# Bentuk data_train

data_train

,Judul,Narasi,Indikator
0,APD dari China Dibeli dengan Uang Pribadi Prabowo,Pada situs kemhan.go.id dalam artikel berjudul...,0
1,Video Penerbangan dari Italia ke ethiopia. Se...,BUKAN di Ethiopia dan TIDAK terkait COVID-19. ...,0
2,Akun Twitter resmi Menteri Pendidikan dan Keb...,BUKAN akun resmi milik Mendikbud Nadiem Makari...,0
3,Inggris Rusuh Akibat Lockdown,Hasil Periksa Fakta Auliyaa Muhammad Hesa (Ang...,0
4,Peta Zona Merah Virus Corona Wilayah Purwokerto,Melalui media sosial Facebook dan pesan berant...,0
...,...,...,...
604,Wanti-wanti Pemerintah Antisipasi Lonjakan Cov...,Presiden Partai Keadilan Sejahtera (PKS) Ahmad...,1
605,Eks Direktur PTPN XI Budi Prabowo Segera Disid...,Logo Komisi Pemberantasan Korupsi di Gedung Ba...,1
606,Pegawai KPI Korban Pelecehan Depresi karena Ka...,Komisi Penyiaran Indonesia (KPI)(Instagram/KPI...,1
607,"Cegah Penyebaran Omicron, Pemerintah Diminta B...",Anggota Komisi IX DPR Netty Prasetiyani.\n (DO...,1


In [35]:
# Jumlah text per-kategori(Indikator)

data_train['Indikator'].value_counts()

0    334
1    274
Name: Indikator, dtype: int64

In [36]:
# Dari cell sebelumnya terlihat jelas bahwa dataset kita sangat tidak balance
# Untuk membuat datasetnya balance

# Pilih dataset dengan Indikator 1 dan lakukan randomisasi untuk setiap baris datanya
false_news = data_train[data_train['Indikator'] == 1].sample(frac=1)

# Concat dataset berIndikator 1 yang telah dipilih dengan dataset berIndikator 0
# dimana jumlah dataset berlabel 1 yang digabungkan sejumlah banyak dataset berIndikator 0 + 200
true_fact = data_train[data_train['Indikator'] == 0]
df = true_fact.append(false_news[:len(true_fact) + 200])

df

,Judul,Narasi,Indikator
0,APD dari China Dibeli dengan Uang Pribadi Prabowo,Pada situs kemhan.go.id dalam artikel berjudul...,0
1,Video Penerbangan dari Italia ke ethiopia. Se...,BUKAN di Ethiopia dan TIDAK terkait COVID-19. ...,0
2,Akun Twitter resmi Menteri Pendidikan dan Keb...,BUKAN akun resmi milik Mendikbud Nadiem Makari...,0
3,Inggris Rusuh Akibat Lockdown,Hasil Periksa Fakta Auliyaa Muhammad Hesa (Ang...,0
4,Peta Zona Merah Virus Corona Wilayah Purwokerto,Melalui media sosial Facebook dan pesan berant...,0
...,...,...,...
371,"Vietnam Mulai Panasi Mesin Piala AFF U-23, Tim...","Vietnam Mulai Panasi Mesin Piala AFF U-23, Tim...",1
588,Aksi Kriminal Libatkan Perempuan dan Anak di J...,Kapolres Metro Jakarta Barat Kombes Pol Ady Wi...,1
344,Wejangan Asnawi Mangkualam untuk Pemain Indone...,Wejangan Asnawi Mangkualam untuk Pemain Indone...,1
481,Kemenkumham Deklarasikan Janji Kinerja 2022,Jajaran Kementerian Hukum dan HAM (Kemenkumham...,1


In [37]:
# Kita akan menggunakan fitur narasi saja dalam melakukan prediksi terhadap Indikator
feature = df['Narasi']
Indikator = df['Indikator']

In [38]:
# Mengubah semua huruf pada setiap baris menjadi huruf kecil dan
# melakukan stemming pada setiap baris
lower = [stemmer.stem(row.lower()) for row in feature]

# Hasil stem dan lower
lower[:5]

['pada situs kemhan go id dalam artikel judul menhan prabowo subianto serah bantu alkes dari tiongkok untuk rumah sakit rujuk tangan covid-19 tulis bahwa operasi humanitarian',
 'bukan di ethiopia dan tidak kait covid-19 video itu adalah simulasi darurat teroris di bandara internasional blaise diagne senegal pada november 2019 lengkap di bagi',
 'bukan akun resmi milik mendikbud nadiem makarim kemendikbud nyata bahwa nadiem makarim tidak milik akun pribadi di media sosial apa saat ini akun nadimmakarim itu',
 'hasil periksa fakta auliyaa muhammad hesa anggota komisariat mafindo ui ukm fc ui unggah video lalu facebook tentang rakyat di inggris yang tindak cara',
 'lalu media sosial facebook dan pesan beranta whatsapp edar unggah yang serta dengan tangkap layar upa peta yang klaim rupa daerah zona merah wilayah purwokerto']

In [39]:
# Melakukan tokenisasi untuk setiap baris dataset
tokens = [word_tokenize(element) for element in lower]

# Hasil tokenisasi setiap baris
tokens[:5]

[['pada',
  'situs',
  'kemhan',
  'go',
  'id',
  'dalam',
  'artikel',
  'judul',
  'menhan',
  'prabowo',
  'subianto',
  'serah',
  'bantu',
  'alkes',
  'dari',
  'tiongkok',
  'untuk',
  'rumah',
  'sakit',
  'rujuk',
  'tangan',
  'covid-19',
  'tulis',
  'bahwa',
  'operasi',
  'humanitarian'],
 ['bukan',
  'di',
  'ethiopia',
  'dan',
  'tidak',
  'kait',
  'covid-19',
  'video',
  'itu',
  'adalah',
  'simulasi',
  'darurat',
  'teroris',
  'di',
  'bandara',
  'internasional',
  'blaise',
  'diagne',
  'senegal',
  'pada',
  'november',
  '2019',
  'lengkap',
  'di',
  'bagi'],
 ['bukan',
  'akun',
  'resmi',
  'milik',
  'mendikbud',
  'nadiem',
  'makarim',
  'kemendikbud',
  'nyata',
  'bahwa',
  'nadiem',
  'makarim',
  'tidak',
  'milik',
  'akun',
  'pribadi',
  'di',
  'media',
  'sosial',
  'apa',
  'saat',
  'ini',
  'akun',
  'nadimmakarim',
  'itu'],
 ['hasil',
  'periksa',
  'fakta',
  'auliyaa',
  'muhammad',
  'hesa',
  'anggota',
  'komisariat',
  'mafindo',
 

In [40]:
# train_test_split digunakan untuk memecah dataset menjadi 2 bagian
# X_train dan y_train mewakili data yang akan dilakukan pada fitting model(Training model)
# X_test dan y_test  mewakili data yang akan dilakukan pada evaluasi model
X_train, X_test, y_train, y_test = train_test_split(tokens, Indikator, test_size=0.1, stratify=Indikator)

In [41]:
# Melihat ukuran data latih dan data uji
print('X_train : ', len(X_train))
print('X_test : ', len(X_test))

X_train :  547
X_test :  61


In [42]:
# Encoder for Data Label
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

y_train

array([0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,

In [43]:
# Fitting dataset terhadap tf-idf
Tfidf_vect.fit(["".join(row) for row in X_train])

TfidfVectorizer()

In [44]:
# Mentransformasikan hasil fitting terhadap data X_train dan X_test
X_train_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_train])
X_test_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_test])

In [45]:
# Classifier - Algoritma - SVM
# fitting/training datasets pada algoritma SVM(Support Vector Machine)
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma="auto", verbose=True)
SVM.fit(X_train_Tfidf, y_train)  # predict the labels on validation dataset

# Menggunakan metrics accuracy untuk melihat performa model
predictions_SVM = SVM.predict(X_test_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100)

[LibSVM]SVM Accuracy Score ->  59.01639344262295


In [46]:
#Algoritma RandomForest

rf = RandomForestClassifier()
rf.fit(X_train_Tfidf, y_train)

prediction_rf = rf.predict(X_test_Tfidf)
print("RandomForest Accuracy Score -> ", accuracy_score(prediction_rf, y_test)*100)

RandomForest Accuracy Score ->  59.01639344262295
